In [1]:
import os
import pandas as pd
import re
import tkinter as tk
from tkinter import Tk
from tkinter import font
from tkinter import filedialog
from PIL import ImageTk, Image
import PIL
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
def damage_data(input_file):
    # X does Y amount of damage to Z
    pattern_damager = r'\]\s.*?\]\s(.*)\sdamages' # '-SOMEONE- damages' ".*?" to get second match ]s 
    pattern_damaged = r'damages\s(.*)\:' # 'damages -SOMEONE-'
    pattern_damage_done = r'\:\s(\d{1,3})\s\(' # 'damages someone: -NUMBER-'

    temp_list = []
    end_list = []

    for line in input_file.readlines():

        #Construct 3-element lists (damager, damaged, damage) and add to list of lists
        if len(temp_list) == 3:
            end_list.append(temp_list)
            temp_list = []
        
        #Extract damager, damaged and damage_done from each line to the temp_list
        if 'damages' in line:
            temp_list.append(re.search(pattern_damager, line).group(1))
            temp_list.append(re.search(pattern_damaged, line).group(1)) 
            temp_list.append(re.search(pattern_damage_done, line).group(1)) 
        else:
            next
    
    return end_list

In [12]:
def generate_damage_table(txt_or_folder):
    
    global damage_df #global var for dataframe, as it's the end result
    
    if os.path.isdir(txt_or_folder): #process in loop with 2 extra fields added to DF if a folder is picked
        damage_df = pd.DataFrame(columns=['damager', 'damaged', 'damage_done', 'date', 'quest'])
        for file in os.listdir(txt_or_folder):
            if file.endswith(".txt"):
                with open(txt_or_folder+'/'+file, 'r') as text_file:
                    try:
                        date = re.search(r'(\d{8})', file).group(1) # extracted from filename 'yyyymmdd' format at start
                        quest = re.search(r'\_(.*)\.txt', file).group(1) # extracted from filename '_zzz.txt' format at end
                    except:
                        date = datetime.now().strftime("%Y%m%d")
                        quest = 'N/A'
                        # warning = 'Incorrect naming format was specified - please use "yyyymmdd_name.txt", with one underscore, for example "20201231_duergar.txt"'

                    end_list = damage_data(text_file) #call the first function here
                    df = pd.DataFrame(end_list, columns=['damager', 'damaged', 'damage_done']).assign(date=date, quest=quest)
                    damage_df = damage_df.append(df, ignore_index=True)
                    text_file.close()
            else:
                next #skip if not txt file

        damage_df = damage_df.astype({'damage_done':'int', 'date':'datetime64'})
        damage_df.sort_values(by=['quest', 'damage_done'], ascending=False, inplace=True)
        
    else: #if not a folder, meaning if standalone txt was picked
        damage_df = pd.DataFrame(columns=['damager', 'damaged', 'damage_done'])
        if txt_or_folder.endswith(".txt"):
            with open(txt_or_folder, 'r') as text_file:
                end_list = damage_data(text_file)
                damage_df = pd.DataFrame(end_list, columns=['damager', 'damaged', 'damage_done'])
                damage_df = damage_df.astype({'damage_done':'int'})
                damage_df.sort_values(by=['damage_done'], ascending=False, inplace=True)
                text_file.close()    

    if len(end_list) != 0: #save to excel if the list was not empty, meaning wrong txt was used
        timestamp = datetime.now().strftime("%Y.%m.%d_%H%M%S")
        damage_df.to_excel(r'./output_files/output_'+timestamp+'.xls', index=False)
    else:
        list_len_error = 'Wrong txt file was selected, no fight details found.'
   
    return damage_df

### Interface with Tkinter

In [9]:
#Functions for the buttons to pick a folder or a txt file

def get_txt():
    global filename
    filename = filedialog.askopenfilename()
    return filename


def get_folder():
    global folder
    folder = filedialog.askdirectory(title='Select Folder')
    return folder

In [ ]:
#Basic UI

app = tk.Tk()

HEIGHT = 500
WIDTH = 889
newsize=(HEIGHT,WIDTH)

C = tk.Canvas(app, height=HEIGHT, width=WIDTH)

#Add background image
try:
    background_image = Image.open(r'./background.gif')
    photo_image = ImageTk.PhotoImage(background_image)
    label = tk.Label(app, image=photo_image)
    label.place(x=0, y=0, relwidth=1, relheight=1)
except:
    None

frame = tk.Frame(app,  bg='#FCF3CF', bd=5)
frame.place(relx=0.5, rely=0.025, relwidth=0.3, relheight=0.1, anchor='n')

get_file_btn = tk.Button(frame, text='Get Txt', font=font.Font(size=10), fg='#FCF3CF', bg='#AF601A', 
                                                        command=lambda: generate_damage_table(get_txt()))
get_file_btn.place(relx=0.1, relheight=1, relwidth=0.35)
get_folder_btn = tk.Button(frame, text='Get Folder', font=font.Font(size=10), fg='#FCF3CF', bg='#AF601A', 
                                                        command=lambda: generate_damage_table(get_folder()))
get_folder_btn.place(relx=0.55, relheight=1, relwidth=0.35)

lower_frame = tk.Frame(app, bg='#641E16', bd=3)
lower_frame.place(relx=0.5, rely=0.6, relwidth=0.75, relheight=0.35, anchor='n')

instr_text ='''
This programme processes NWN log files and saves the output in the "output_files" folder in excel format.

By clicking the "Get Txt" button and selecting a txt log file, you can process a standalone file.

By clicking the "Get Folder" button and selecting a folder, you can process multiple files at once and
also add two extra columns. The recommended file naming convention is "yyyymmdd_quest.txt", for
example "20201231_drider.txt"
'''

instructions = tk.Label(lower_frame, anchor='nw', justify='left', text=instr_text, wraplengt=600)
instructions.config(font=font.Font(size=10), fg='#AF601A', anchor="center", bg='#FCF3CF')
instructions.place(relwidth=1, relheight=1)

C.pack()

app.mainloop()

In [ ]:
damage_df

In [ ]:
damage_df.groupby(['damager']).sum().sort_values('damage_done', ascending=False)